In [1]:
import requests

res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')

with open('nsmc_train.csv', 'wb') as f:
    f.write(res.content)

In [2]:
import pandas as pd
nsmc = pd.read_csv('nsmc_train.csv', sep='\t')

In [3]:
nsmc.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
import stanza

In [5]:
nlp = stanza.Pipeline('ko')

2021-03-26 13:26:26 INFO: Loading these models for language: ko (Korean):
| Processor | Package |
-----------------------
| tokenize  | kaist   |
| pos       | kaist   |
| lemma     | kaist   |
| depparse  | kaist   |

2021-03-26 13:26:27 INFO: Use device: gpu
2021-03-26 13:26:27 INFO: Loading: tokenize
2021-03-26 13:26:30 INFO: Loading: pos
2021-03-26 13:26:30 INFO: Loading: lemma
2021-03-26 13:26:30 INFO: Loading: depparse
2021-03-26 13:26:31 INFO: Done loading processors!


In [6]:
text = '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'

In [7]:
doc = nlp(text)

In [19]:
def extract_nouns(text):
    doc = nlp(text)
    for sentence in doc.sentences:
        for word in sentence.words:
            lemma = word.lemma.split('+')
            xpos = word.xpos.split('+')
            for w, p in zip(lemma, xpos):
                if p.startswith('n'):
                    yield w

In [20]:
list(extract_nouns(text))

['교도소', '이야기구먼', '..솔직히', '재미', '.평점', '조정']

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
cv = CountVectorizer(max_features=100, tokenizer=extract_nouns)

In [25]:
tdm = cv.fit_transform(nsmc.loc[0:100, 'document'])

In [26]:
tdm.shape

(101, 100)

In [27]:
word_count = pd.DataFrame({
    '단어': cv.get_feature_names(),
    '빈도': tdm.sum(axis=0).flat
})

In [28]:
word_count.sort_values('빈도', ascending=False).head()

,단어,빈도
47,영화,19
46,연기,6
49,완전,6
9,내용,5
92,진짜,5


In [29]:
word_count.to_excel('nsmc-count.xlsx')